# cs542 Machine Learning

Collaborator: Yiteng Xu, Changlei Shi, ChenCheng Wang, Yi Li

# Pre-process data

In [2]:
import csv
import pandas as pd
import numpy as np
import json
from collections import defaultdict

def readJson(f):
    for l in open(f):
        yield eval(l)

In [19]:
items = []

for l in open("anonymous-msweb.data"):
    if l.startswith("A"):
        m,n,p,q,i = l.strip().split(',')
        q = q[1:-1]
        i = i[1:-1]
        tmp = [n, q, i]
        items.append(tmp)

items_df = pd.DataFrame(items)
items_df.to_csv("item.txt")

In [27]:
current = ""
item = ""
user_item = []
training_user_item_dict = {}

for l in open("anonymous-msweb.data"):
    if l.startswith("C"):
        m,n,p = l.strip().split(',')
        current = p
    if l.startswith("V"):
        a,b,c = l.strip().split(',')
        item = b
        tp = [current, item]
        user_item.append(tp)
        if current in training_user_item_dict:
            cur_list = training_user_item_dict[current]
            cur_list.append(item)
            training_user_item_dict[current] = cur_list
        else:
            my_list = [item]
            training_user_item_dict[current] = my_list
user_item_df = pd.DataFrame(user_item)
user_item_df.to_csv("user_item.txt")

json_str = json.dumps(training_user_item_dict)
data = json.loads(json_str)
with open('user_item_dict.json', 'w') as f:
    json.dump(data, f)
#print training_user_item_dict
'''
w = csv.writer(open("user_item_dict.csv", "w"))
for key, val in training_user_item_dict.items():
    w.writerow([key, val])
'''

'\nw = csv.writer(open("user_item_dict.csv", "w"))\nfor key, val in training_user_item_dict.items():\n    w.writerow([key, val])\n'

In [26]:
import json
items = []

for l in open("anonymous-msweb.test"):
    if l.startswith("A"):
        m,n,p,q,i = l.strip().split(',')
        q = q[1:-1]
        i = i[1:-1]
        tmp = [n, q, i]
        items.append(tmp)

items_df = pd.DataFrame(items)
items_df.to_csv("item_test.txt")

current = ""
item = ""
user_item = []
training_user_item_dict = {}
for l in open("anonymous-msweb.test"):
    if l.startswith("C"):
        m,n,p = l.strip().split(',')
        current = p
    if l.startswith("V"):
        a,b,c = l.strip().split(',')
        item = b
        tp = [current, item]
        user_item.append(tp)
        if current in training_user_item_dict:
            cur_list = training_user_item_dict[current]
            cur_list.append(item)
            training_user_item_dict[current] = cur_list
        else:
            my_list = [item]
            training_user_item_dict[current] = my_list
user_item_df = pd.DataFrame(user_item)
user_item_df.to_csv("user_item_test.txt")

#print training_user_item_dict
json_str = json.dumps(training_user_item_dict)
data = json.loads(json_str)
with open('user_item_dict_test.json', 'w') as f:
    json.dump(data, f)

'''
w = csv.writer(open("user_item_dict_test.csv", "w"))
for key, val in training_user_item_dict.items():
    w.writerow([key, val])
'''

In [23]:
import csv
dict = {}
for key, val in csv.reader(open("user_item_dict_test.csv")):
    print key
    val[1:-1].replace('\'', '')
    print str(val[1:-1]).split(",")
    break

14478
["'1136'", " '1046'", " '1018'"]


# Implementing Memory-based Algorithms

In [102]:
import csv
import pandas as pd
import numpy as np
import json
from collections import defaultdict

#load file
user_item_df = pd.read_csv("user_item.txt")
user_item_df.columns = ['index','user', 'item']
user_item_df.drop('index', axis=1, inplace=True)

#group by user and get the total vote of each user
tmp = user_item_df.groupby(['user']).count()
user_votes = []
for index, item in tmp.iterrows():
    c = int(item['item'])
    tp = [index, c]
    user_votes.append(tp)

#save results to another dataframe
user_votes_df = pd.DataFrame(user_votes)
user_votes_df.columns = ['user','count']

#print the mean of votes per user
print user_votes_df['count'].mean()
#print number of users in trainning set
print len(user_votes_df)


3.01592736388
32711


In [4]:
import math
from collections import OrderedDict
from operator import itemgetter
import csv
import pandas as pd
import numpy as np
import json
from collections import defaultdict
    
def prediction(user_a, item_j, compare_users_data, alpha):
    tmp = {}
    for user_i in compare_users_data:
        _list = compare_users_data[user_i]
        if user_a is not user_i and len(_list) > alpha:
            v_ij = 0
            if item_j in _list:
                v_ij = 1
            w = getWeight2(user_a, user_i, compare_users_data)
            tmp[user_i] = w
    
    sorted_tmp = sorted(tmp.iteritems(), key=itemgetter(1), reverse=True)
    nearest_neighbour = [sorted_tmp[0][0], sorted_tmp[1][0],sorted_tmp[2][0],sorted_tmp[3][0],sorted_tmp[4][0],sorted_tmp[5][0],sorted_tmp[6][0],sorted_tmp[7][0],sorted_tmp[8][0]]
    
    W = 0
    total = 0
    big_list = []
    for user in nearest_neighbour:
        _list = compare_users_data[user]
        big_list = big_list + _list
    if item_j in list(set(big_list)):
        return 1
    else:
        return 0

In [5]:
import json
import math

def getWeight(a, i, compare_users_data):
    set1 = set(compare_users_data[a])
    set2 = set(compare_users_data[i])
    length1 = len(set1)
    length2 = len(set2)
    inter = set1.intersection(set2)
    den1 = math.sqrt(length1)
    den2 = math.sqrt(length2)
    res = float(len(inter)) / (den1 * den2)
    return res

def getWeight2(a, i, compare_users_data):
    set1 = set(compare_users_data[a])
    set2 = set(compare_users_data[i])
    union = set1.union(set2)
    inter = set1.intersection(set2)
    res = float(len(inter)) / len(union)
    return res

with open('user_item_dict.json') as data_file:    
    compare_users_data = json.load(data_file)

print prediction("10077","1004",compare_users_data,4)


1


In [7]:
def KNNTest(alpha):
    f_file = "user_item_test.txt"
    total = 0
    count = 0
    for l in open(f_file):
        if l.startswith(","):
            continue
        a = l.split(",")
        user = a[1]
        item = a[2].rstrip('\n')
        total = total + 1
        res = prediction(user, item,compare_users_data,alpha)
        if res == 1:
            count = count + 1
        
    return float(count) / total

print 'Alpha is 4: '
print KNNTest(4)
print '\n'

print 'Alpha is 6: '
print KNNTest(6)
print '\n'

print 'Alpha is 9: '
print KNNTest(9)
print '\n'

Alpha is 4: 
0.528141662827


Alpha is 6: 
0.624580343625


Alpha is 9: 
0.715621091436




Code for setting the style of the notebook

# Test for Bayesian Models classifer

In [15]:
import pandas as pd
import numpy as np
import json
'''
all_item_df = pd.read_csv("item.txt")
all_item_df.columns = ["index", "itemId", "description", "url"]
all_item_df.drop(['index', 'url', 'description'], axis=1, inplace=True)
result = all_item_df.sort(['itemId'], ascending=[True])
numpyMatrix = result.as_matrix()
flat = [x for sublist in numpyMatrix for x in sublist]
print flat
'''
count = 1000
columnName = []
while count <= 1297:
    columnName.append(count)
    count = count + 1

with open('user_item_dict.json') as data_file:    
    compare_users_data = json.load(data_file)
    
big_matrix = []
index = []
for attribute, value in compare_users_data.iteritems():
    index.append(attribute)
    tmp_arr = np.zeros(298)
    for element in value:
        #print element
        tmp_arr[int(element) - 1000] = 1
    big_matrix.append(tmp_arr)

    #print attribute, value
#print index
big_matrix_df = pd.DataFrame(big_matrix)
big_matrix_df.columns = columnName
big_matrix_df.index = index
#print index[0]
#big_matrix_df = big_matrix_df.set_index(index)
print big_matrix_df

#big_matrix_df.columns = flat
#print big_matrix_df


11542


KeyError: u'11542'

In [1]:

from IPython.core.display import HTML
def css_styling():
    styles = open("../../theme/custom.css", "r").read()
    return HTML(styles)
css_styling()

IOError: [Errno 2] No such file or directory: '../../theme/custom.css'